# DMQC Report: WMO 4902410

## Current status and metadata: 

- MetOcean NOVA float deployed in the Lab Sea
- Deployed in April 2017, inactive as of 2021
- Completed 126 cycles, though float reports 257 cycles (?)
- No in-air data
- Core files in D mode, B files in R mode

In [1]:
import warnings
warnings.filterwarnings('ignore')

from matplotlib.dates import datestr2num
import matplotlib.pyplot as plt

from netCDF4 import Dataset

from pathlib import Path
import numpy as np
import pandas as pd
import bgcArgoDMQC as bgc

wmo = 4902410 # insert WMO number here
fig_path = Path('../../../figures/') / f'{wmo}' # where to save figures
fig_path.mkdir(exist_ok=True)
bgc.io.get_argo(wmo, local_path=bgc.io.Path.ARGO_PATH) # download the data to ensure you have up to date files
flt = bgc.sprof(wmo)
flt.clean()
gains = flt.calc_gains(ref='WOA') # calculate gain, can change to ref='NCEP' if in-air data is available
print(np.nanmean(gains))

# check what/if anything is done already
nc = Dataset(flt.__Sprof__)
print(bgc.io.read_ncstr(nc['SCIENTIFIC_CALIB_COMMENT'][:][50,-1,3,:]))
print(nc['DOXY_ADJUSTED'][:][-1,:]/nc['DOXY'][:][-1,:])
nc.close()

\Users\GordonC\Documents\data\Argo\dac\meds\4902410\4902410_BRtraj.nc
\Users\GordonC\Documents\data\Argo\dac\meds\4902410\4902410_Rtraj.nc
\Users\GordonC\Documents\data\Argo\dac\meds\4902410\4902410_Sprof.nc
\Users\GordonC\Documents\data\Argo\dac\meds\4902410\4902410_meta.nc
\Users\GordonC\Documents\data\Argo\dac\meds\4902410\4902410_prof.nc
\Users\GordonC\Documents\data\Argo\dac\meds\4902410\4902410_tech.nc
hello POSITION_QC
hello PRES_QC
hello PRES_ADJUSTED_QC
hello TEMP_QC
hello TEMP_ADJUSTED_QC
hello PSAL_QC
hello PSAL_ADJUSTED_QC
hello DOXY_QC
hello DOXY_ADJUSTED_QC
hello O2Sat_QC
Extracting WOA data for Jan
Extracting WOA data for Feb
Extracting WOA data for Mar
Extracting WOA data for Apr
Extracting WOA data for May
Extracting WOA data for Jun
Extracting WOA data for Jul
Extracting WOA data for Aug
Extracting WOA data for Sep
Extracting WOA data for Oct
Extracting WOA data for Nov
Extracting WOA data for Dec
Bounding climatological profile(s) missing data - taking simple average

## DOXY Audit Status

- If the float has any profiles present in the DOXY audit, which are they are why?
- Likely will usually be left empty/deleted

In [3]:
audit_file = list(Path('../../../audit/').glob('DOXY_WOA*'))[-1]
df = pd.read_csv(audit_file, sep='\t', header=25)
df = df.loc[df.WMO == wmo]
df['date'] = [datestr2num(t) for t in df['profile date']]
print(df['WOA G_raw'].mean())

nan


## Visual QC

- Visualize the float and reference surface saturations, as well as the profiles. 
- Observe if DOXY data is set to '3' as it should be or if it is '1' or '2'. 
- Try to note cycles or other identifying features of potentially problematic profiles.

In [4]:
g = flt.plot('gain', ref='WOA')
g.axes[0].set_title(f'WMO: {wmo}', loc='left', fontweight='bold')
g.axes[0].plot(flt.df.SDN.loc[flt.df.PRES < 50], flt.df.O2Sat.loc[flt.df.PRES < 50], 'o', zorder=0, alpha=0.1)
g.fig.savefig(fig_path / 'gain_initial.png', bbox_inches='tight', dpi=250)

Figure 1: Top panel: Float oxygen percent saturation (blue line) compared to WOA percent saturation (orange line), with DOXY audit flagged cycle percent saturation (green stars, if applicable) and raw float percent saturation (orange circles). Bottom panel: calculated gains (blue dots) and flagged gains from DOXY audit (orange stars).

Notes: [notes on what this figure tells you here] 

In [5]:
flt.reset()
g = flt.plot('qcprofiles', varlist=['DOXY', 'DOXY_ADJUSTED', 'O2Sat', 'TEMP', 'PSAL'])
g.fig.savefig(fig_path / 'qcprofiles.png', bbox_inches='tight', dpi=250)
g = flt.plot('qcprofiles', varlist=['DOXY', 'DOXY_ADJUSTED', 'O2Sat', 'TEMP', 'PSAL'])
g.axes[0].set_ylim((200,0))
g.fig.savefig(fig_path / 'qcprofiles_shallow.png', bbox_inches='tight', dpi=250)

In [7]:
flt.df.loc[flt.df.DOXY < 120]

PRES  PRES_QC    TEMP  TEMP_QC       PSAL  PSAL_QC  DOXY_QC  \
55491   390.000000      1.0  48.562      4.0  16.282000      4.0      1.0   
60629   390.000000      1.0  48.562      4.0  16.282000      4.0      3.0   
63047  1469.300049      1.0   4.020      1.0  35.021000      1.0      3.0   
63048  1474.900024      1.0   4.014      1.0  35.061001      4.0      3.0   
63049  1481.099976      1.0   4.018      1.0  35.354000      4.0      3.0   
63550  1364.699951      1.0   4.333      1.0  35.050999      4.0      4.0   
63551  1370.099976      1.0   4.319      1.0  35.071999      4.0      3.0   
63552  1375.300049      1.0   4.307      1.0  35.113998      4.0      3.0   
63553  1380.099976      1.0   4.289      1.0  35.341000      4.0      3.0   
63554  1383.599976      1.0   4.297      1.0  35.773998      4.0      3.0   
64129  1639.800049      1.0   4.102      1.0  35.035000      1.0      4.0   
64130  1644.599976      1.0   4.096      1.0  35.044998      1.0      4.0   
64131  1649.500000      1.0   4.096      1.0  35.049999      1.0      3.0   
64132  1655.000000      1.0   4.088      1.0  35.084000      4.0      3.0   
64133  1660.099976      1.0   4.097      1.0  35.134998      4.0      3.0   
64134  1665.400024      1.0   4.095      1.0  35.372002      4.0      3.0   
64135  1669.000000      1.0   4.098      1.0  35.647999      4.0      3.0   
64649  1625.000000      1.0   4.218      1.0  35.047001      1.0      3.0   
64650  1629.800049      1.0   4.206      1.0  35.047001      1.0      4.0   
64651  1635.099976      1.0   4.189      1.0  35.053001      1.0      4.0   
64652  1639.599976      1.0   4.164      1.0  35.075001      4.0      3.0   
64653  1645.099976      1.0   4.156      1.0  35.117001      4.0      3.0   
64654  1650.500000      1.0   4.142      1.0  35.258999      4.0      3.0   
64655  1655.599976      1.0   4.145      1.0  35.832001      4.0      3.0   
65242  1975.300049      1.0   3.771      1.0  35.008999      1.0      3.0   
65243  1980.199951      1.0   3.768      1.0  35.012001      1.0      3.0   
65244  1984.400024      1.0   3.765      1.0  35.016998      1.0      3.0   
65245  1989.699951      1.0   3.759      1.0  35.025002      4.0      3.0   
65246  1994.599976      1.0   3.756      1.0  35.041000      4.0      3.0   
65247  2000.000000      1.0   3.755      1.0  35.081001      4.0      3.0   
65248  2005.900024      1.0   3.756      1.0  35.328999      4.0      3.0   
65249  2008.500000      1.0   3.757      1.0  35.516998      4.0      3.0   
65728  1809.199951      1.0   3.958      1.0  35.033001      1.0      3.0   
65729  1815.000000      1.0   3.970      1.0  35.036999      1.0      4.0   
65730  1820.199951      1.0   3.976      1.0  35.041000      1.0      3.0   
65731  1824.900024      1.0   3.966      1.0  35.069000      1.0      3.0   
65732  1830.300049      1.0   3.962      1.0  35.136002      1.0      3.0   
65733  1836.000000      1.0   3.965      1.0  35.518002      1.0      3.0   
65734  1838.400024      1.0   3.967      1.0  35.793999      1.0      3.0   

       DOXY_ADJUSTED_QC  DOXY_ADJUSTED        DOXY  ...  N_PROF  \
55491              -1.0        99999.0   94.274940  ...   106.0   
60629              -1.0        99999.0   94.274940  ...   116.0   
63047              -1.0        99999.0  105.968369  ...   120.0   
63048              -1.0        99999.0   85.423233  ...   120.0   
63049              -1.0        99999.0   76.140030  ...   120.0   
63550              -1.0        99999.0  112.395432  ...   121.0   
63551              -1.0        99999.0   84.993561  ...   121.0   
63552              -1.0        99999.0   64.599503  ...   121.0   
63553              -1.0        99999.0   56.688965  ...   121.0   
63554              -1.0        99999.0   55.116943  ...   121.0   
64129              -1.0        99999.0   95.202400  ...   122.0   
64130              -1.0        99999.0   67.350624  ...   122.0   
64131              -1.0        99999.0   39.971256  ...   122.

Figures 2,3: Profiles coloured by QC flag (1-2, green, 3, yellow, 4, red) on difference depth scales.

Notes: [notes on what this figure tells you here] 

## Flag bad data, recalculate gain

- Using the plots above (and likely some more specific plots, made on a case-by-case basis), flag bad DOXY data as 4. 
- After flagging bad data, re-clean the data and re-calculate the gains.

In [ ]:
bad_oxygen = [] # index defining where the oxygen should be flagged as bad
flt.update_field('DOXY_QC', 4, where=bad_oxygen)
flt.update_field('DOXY_ADJUSTED_QC', 4, where=bad_oxygen)
flt.clean()
new_gains = flt.calc_gains(ref='WOA')
print(np.nanmean(new_gains))

flt.reset()
g = flt.plot('qcprofiles', varlist=['DOXY', 'DOXY_ADJUSTED', 'O2Sat', 'TEMP', 'PSAL'])
g.fig.savefig(fig_path / 'qcprofiles_postqc.png', bbox_inches='tight', dpi=250)
g = flt.plot('qcprofiles', varlist=['DOXY', 'DOXY_ADJUSTED', 'O2Sat', 'TEMP', 'PSAL'])
g.axes[0].set_ylim((200,0))
g.fig.savefig(fig_path / 'qcprofiles_shallow_postqc.png', bbox_inches='tight', dpi=250)

flt.clean()
g = flt.plot('gain', ref='WOA')
g.axes[0].set_title(f'WMO: {wmo}', loc='left', fontweight='bold')
g.axes[0].plot(flt.df.SDN.loc[flt.df.PRES < 50], flt.df.O2Sat.loc[flt.df.PRES < 50], 'o', zorder=0, alpha=0.1)
g.fig.savefig(fig_path / 'gain_final.png', bbox_inches='tight', dpi=250)

Figure x: Repeated gain plot with bad data removed.

## Update Fields and Export

- DOXY should be 3 by default, but many old floats have 1 or 0. Though it will be redundant in some cases, make sure DOXY_QC is not good.
- Apply the gain to calculate DOXY_ADJUSTED. 
- Fill in DOXY_ADJUSTED_ERROR with an appropriate fixed value.
- Fill in DOXY_ADJUSTED_QC. 
- Make sure DOXY_ADJUSTED and DOXY_ADJUSTED_ERROR are `FillValue` wherever DOXY_ADJUSTED_QC is bad.
- Export the D-mode files. During export, HISTORY parameters will be filled as well as SCIENTIFIC_CALIB parameters using some default structures and user info set by calling `bgc.configure.configure()`.

In [ ]:
# make sure DOXY_QC is 3 instead of 0,1,2
flt.update_field('DOXY_QC', 3, where=flt.DOXY_QC.isin([0, 1, 2]))
# fill in DOXY_ADJUSTED_QC appropriately
# flt.update_field('DOXY_ADJUSTED_QC', 4, where=flt.TEMP_ADJUSTED_QC == 4)
# flt.update_field('DOXY_ADJUSTED_QC', 3, where=flt.PSAL_ADJUSTED_QC == 4)
# make sure the data is FillValues where adjusted values are still bad
flt.update_field('DOXY_ADJUSTED_QC', 4, where=flt.DOXY_ADJUSTED.isna())
flt.set_fillvalue('DOXY_ADJUSTED', where=flt.DOXY_ADJUSTED_QC == 4)
flt.set_fillvalue('DOXY_ADJUSTED_ERROR', where=flt.DOXY_ADJUSTED_QC == 4)

In [ ]:
# export the files - version for if already in A mode and unchanged
# comment = 'by Henry Bittig; ORCID 0000-0002-8621-3095; after Bittig et al. (2018) eq. 23 T-dependent slope on PPOX_DOXY with linear time trend in slope; Using in-air obs; No time response correction done; error gives 1 sigma'
# comment += f"; Visual QC by Christopher Gordon (DFO, ORCID {bgc.config['orcid']})"
# comment = comment.replace('No time response correction done;', '')
# flt.export_files(data_mode='D', comment=comment, coeff='previous', equation='previous')

In [ ]:
# export the files - for R to D mode
# flt.export_files(data_mode='D')

In [ ]:
from pathlib import Path
import pandas as pd
from netCDF4 import Dataset
import copy

wmo = 4900874
loc = Path(f'/Users/GordonC/Documents/data/Argo/dac/meds/D/{wmo}/profiles/')
files = pd.read_csv(Path(f'../../../checker/summary/{wmo}/files.txt'))
varname = 'DOXY'
for fn in files.files:
    print(fn)
    nc = Dataset(loc / fn, 'r+')
    print(nc['DOXY'][:])
    flags = copy.deepcopy(nc[varname+'_QC'][:])
    flags[nc[varname][:].mask] = b'9'
    nc[varname+'_QC'][:] = flags
    if varname == 'DOXY':
        flags = copy.deepcopy(nc[varname+'_ADJUSTED_QC'][:])
        flags[nc[varname][:].mask] = b'9'
        nc[varname+'_ADJUSTED_QC'][:] = flags
    profile = copy.deepcopy(nc['PROFILE_DOXY_QC'][:])
    profile[0] = b'A'
    nc['PROFILE_DOXY_QC'][:] = profile
    nc.close()